# HW4 - Convert WikiHow to PDDL

In this homework we will translate a wikHow article into the Planning Domain Definition Language (PDDL).    The goals of this homework are to:
1. Deepen our understanding of classical planning
2. Understand the process of manually translating from natural language instructions into PDDL.
3. Think about how we might teach GPT-3 to perform the translation automatically (don't worry – you don't have to implement the automatic translation in this homework).
4. Capture some data that might be useful for fine-tuning GPT-3 to convert to PDDL (for possible use in some students' term projects).  

This colab notebook will give you an example of how to convert one step from a wikiHow article into PDDL.  You'll then do this on your own and submit a domain.pddl and several problem.pddl files.

In [ ]:
# Install the PDDL parser
!git clone https://github.com/pucrs-automated-planning/pddl-parser
!cd pddl-parser

Cloning into 'pddl-parser'...
remote: Enumerating objects: 337, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 337 (delta 38), reused 55 (delta 21), pack-reused 260
Receiving objects: 100% (337/337), 79.03 KiB | 851.00 KiB/s, done.
Resolving deltas: 100% (192/192), done.


In [ ]:
# Mount your Google Drive so that you can save your PDDL files.
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/My Drive/CIS 700/HW4'

Mounted at /content/drive


These are some helper functions for loading and saving files, and for displaying long texts.

In [ ]:
import os
import json
import textwrap


## Create a file from the contents of a string. 
def write_to_file(string, path, filename):
  """This is a helper function to create a file from a string."""
  if not os.path.exists(path) and path != "":
    os.makedirs(path)
  with open(os.path.join(path, filename), "w") as f:
      f.write(string)


# Load a JSON file given a filename
# If the file doesn't exist, then return an empty dictionary instead
def load_json(filename):
    try:
        with open(filename, 'r') as f:
            return json.load(f)
    except FileNotFoundError:
        return {}

# Save a JSON file given a full path filename and a dictionary
# If the path doesn't exist, create the directories before saving the file.
def save_json(filename, data):
    if not os.path.exists(os.path.dirname(filename)):
        os.makedirs(os.path.dirname(filename))
    with open(filename, 'w') as f:
        json.dump(data, f, indent=4)


def wrap_text(text):
  wrapped_text = ""
  lines = text.strip().split("\n")

  for line in lines:
    if len(line) > 90:
        w = textwrap.TextWrapper(width=90, break_long_words=False)
        line = '\n'.join(w.wrap(line))
    wrapped_text += line + "\n"
  return wrapped_text


### Step 1: Pick an Interesting wikiHow Article

We'll create our PDDL files from a wikiHow article.  The goal for this is to start from something that describes proceedures and actions and is written in natural language, and then to manually translate it into the description language used for automated planning.

Here are a few wikiHow articles that I thought might be interesting since they had some elements that could make for compelling interactive fiction.  It's fine to pick your own article.   We won't translate the whole article, just a few steps, so you can pick out the parts that you think are most relevant / easiest to create action schema from.

Survival Stories
* [How to Survive in the Woods](https://www.wikihow.com/Survive-in-the-Woods)
* [How to Survive in the Jungle](https://www.wikihow.com/Survive-in-the-Jungle)
* [How to Survive on a Desert Island](https://www.wikihow.com/Survive-on-a-Desert-Island
) 
* [How to Survive on a Deserted Island With Nothing](https://www.wikihow.com/Survive-on-a-Deserted-Island-With-Nothing)
* [How to Get Out of Quicksand](https://www.wikihow.com/Get-Out-of-Quicksand)
* [How to Open a Coconut](https://www.wikihow.com/Open-a-Coconut)
* [How to Test if a Plant Is Edible](https://www.wikihow.com/Test-if-a-Plant-Is-Edible)
* [How to Find True North Without a Compass](https://www.wikihow.com/Find-True-North-Without-a-Compass)
* [How to Survive a Wolf Attack](https://www.wikihow.com/Survive-a-Wolf-Attack)

Kid Detectives
* [How to Make a Detective Kit](https://www.wikihow.com/Make-a-Detective-Kit)
* [How to Disguise Yourself](https://www.wikihow.com/Disguise-Yourself)
* [How to Make a Hidden Camera](https://www.wikihow.com/Make-a-Hidden-Camera)
* [How to Hide Money](https://www.wikihow.com/Hide-Money)
* [How to Spy on People](https://www.wikihow.com/Spy-on-People)
* [How to Hack](https://www.wikihow.com/Hack)
* [How to Make a Grappling Hook](https://www.wikihow.com/Make-a-Grappling-Hook)
* [How to Open a Locked Door](https://www.wikihow.com/Open-a-Locked-Door)
* [How to Create a Secret Society](https://www.wikihow.com/Create-a-Secret-Society)
* [How to Win Fights at School](https://www.wikihow.com/Win-Fights-at-School)

Dystopian Futures
* [How to Survive a Comet Hitting Earth](https://www.wikihow.com/Survive-a-Comet-Hitting-Earth)
* [How to Survive an EMP](https://www.wikihow.com/Survive-an-EMP)
* [How to Survive a Nuclear Attack](https://www.wikihow.com/Survive-a-Nuclear-Attack)
* [How to Build a Fallout Shelter](https://www.wikihow.com/Build-a-Fallout-Shelter)
* [How to Survive a Riot](https://www.wikihow.com/Survive-a-Riot)
* [How to Survive Under Martial Law](https://www.wikihow.com/Survive-Under-Martial-Law)
* [How to Avoid Danger During Civil Unrest](https://www.wikihow.com/Avoid-Danger-During-Civil-Unrest)
* [How to Thwart an Abduction Attempt](https://www.wikihow.com/Thwart-an-Abduction-Attempt)
* [How to Make Papyrus](https://www.wikihow.com/Make-Papyrus)


As an example, I'll pick the [How to Survive in the Woods](https://www.wikihow.com/Survive-in-the-Woods) article, and work on translating Part 1, Step 1 into PDDL.  Here is step 1 from that article: 


<center>
<img src="https://www.wikihow.com/images/thumb/c/c0/Survive-in-the-Woods-Step-11-Version-5.jpg/aid31352-v4-728px-Survive-in-the-Woods-Step-11-Version-5.jpg.webp" class="img-responsive" alt="Search for a source of fresh water. (Creative Commons License)"/>
</center>



> ### Finding Drinking Water
> Search for a source of fresh water.  The first thing that you'll need in order to survive in the woods is water that you can drink. Look for signs of fresh water nearby like areas of green foliage that indicate water is nearby, low-lying areas where water could be collected, and signs of wildlife like animal tracks. It could mean that a creek, stream, or pond is nearby. While finding water is important for survival, be aware some water sources will not be safe - if possible treat all drinking water before using it. 
If there are mountains nearby, look for water collected at the foot of the cliffs.
> * The presence of insects like mosquitoes and flies means that water is nearby.
> * Water from heavily oxygenated water (such as from a big waterfall or rapids) typically is safer than that from a slow or still water source.
> * Freshwater springs are typically safer water sources, although these can be contaminated by mineral or bacteria as well.
> * Remember that all untreated water must be considered risky unless treated. Even crystal clear water can harbor diseases and be dangerous if consumed.

# Homework Assignment Start

We decided to implement [How to Survive in the Woods](https://www.wikihow.com/Survive-in-the-Woods).

## Domain definition

In [ ]:
domain = """
(define (domain survive_in_the_woods)
   (:requirements :strips :typing)
   (:types
       water - item 
       tree - item
       branch - item 
       branches - item
       twigs - item
       leaves - item
       beam - item
       frame - item
       shelter - item
       flowers - item
       wire - item
       snare - item
       bar - item
       player direction location
   )

   (:predicates
      (has_water_source ?loc - location) ; this location has a source of fresh water.
      (treated ?water - water) ; True if the water has been decontaimated by boiling it
      (at ?obj - object ?loc - location) ; an object is at a location 
      (inventory ?player - player ?item - item) ; an item is in the player's inventory
      (connected ?loc1 - location ?dir - direction ?loc2 - location) ; location 1 is connected to location 2 in the direction
      (blocked ?loc1 - location ?dir - direction ?loc2 - location) ; the connection between location 1 and 2 in currently blocked
      (dry ?loc - location) ; this location is dry
      (flat ?loc - location) ; this location is flat
      (has_split_in_trunk ?tree - tree) ; True if this tree has split in its trunk
      (long ?b - branch) ; True if the branch is 10 feet long
      (thick ?b - branch) ; True if the branch is 6 inches thick
      (has_bug ?item - item) ; True if the branch or leaves has bugs
      (edible ?item - item) ; True if the item is edible
      (unwashed ?item - item) ; True if the item has not been washed off 
      (hungry ?p - player) ; True if the player is hungry
      (made_by_animal ?l - location) ; True if this is a small, beaten path that made by an animal
      (hanged ?s - snare) ; True if this snare is hanged


   )

   (:action go ; navigate to an adjacent location 
      :parameters (?dir - direction ?p - player ?l1 - location ?l2 - location) 
      :precondition (and (at ?p ?l1) (connected ?l1 ?dir ?l2) (not (blocked ?l1 ?dir ?l2)))
      :effect (and (at ?p ?l2) (not (at ?p ?l1)))
   )

   (:action get ; pick up an item and put it in the inventory
      :parameters (?item - item ?p - player ?l1 - location) 
      :precondition (and (at ?p ?l1) (at ?item ?l1))
      :effect (and (inventory ?p ?item) (not (at ?item ?l1)))
   )

   (:action get_water ; get water from a location that has a water source like a lake.
      :parameters (?p - player ?loc - location ?water - water) 
      :precondition (and (at ?p ?loc) (has_water_source ?loc))
      :effect (and (inventory ?p ?water) (not (treated ?water)))
   )

   (:action drop ; drop down an item and remove it from the inventory
      :parameters (?item - item ?p - player ?l1 - location) 
      :precondition (and (at ?p ?l1) (inventory ?p ?item) (not (at ?item ?l1)))
      :effect (and (not (inventory ?p ?item)) (at ?item ?l1))
   )

   (:action clear_spider ; clear off any critters or spiders that may be on the branch or leaves
      :parameters (?item - item ?p - player) 
      :precondition (and (inventory ?p ?item) (has_bug ?item))
      :effect (and (not (has_bug ?item)) (inventory ?p ?item))
   )

   (:action insert_beam ; insert end(s) of the branch into the split(s) of the tree(s)
      :parameters (?b - branch ?t - tree ?p - player ?l1 - location ?beam - beam) 
      :precondition (and (at ?p ?l1) (dry ?l1) (flat ?l1) (at ?t ?l1) (has_split_in_trunk ?t) (inventory ?p ?b) (not (has_bug ?b)) (long ?b) (thick ?b) )
      :effect (and (at ?p ?l1) (not (inventory ?p ?b)) (at ?beam ?l1))
   )

   (:action lean_branches ; Lean branches against the main branch to form the structure of your shelter.
      :parameters (?branches - branches ?p - player ?l1 - location ?beam - beam ?frame - frame) 
      :precondition (and (at ?p ?l1) (inventory ?p ?branches) (at ?beam ?l1))
      :effect (and (at ?p ?l1) (not (inventory ?p ?branches)) (at ?frame ?l1))
   )

   (:action layer ; Lay twigs and leaves
      :parameters (?twigs - twigs ?leaves - leaves ?p - player ?l1 - location ?frame - frame ?shelter - shelter) 
      :precondition (and (at ?p ?l1) (inventory ?p ?twigs) (inventory ?p ?leaves) (not (has_bug ?leaves)) (at ?frame ?l1))
      :effect (and (at ?p ?l1) (not (inventory ?p ?twigs)) (not (inventory ?p ?leaves)) (at ?shelter ?l1))
   )

   (:action wash ; Wash off plants that you find.
      :parameters (?i - item ?p - player ?water - water) 
      :precondition (and (inventory ?p ?i) (inventory ?p ?water) (unwashed ?i))
      :effect (and (inventory ?p ?i) (not (unwashed ?i)))
   )

   (:action eat ; Eat anything you find edible
      :parameters (?i - item ?p - player) 
      :precondition (and (hungry ?p) (inventory ?p ?i) (not (unwashed ?i)) (edible ?i))
      :effect (and (not (inventory ?p ?i)) (not (hungry ?p)))
   )

   (:action make_snare ; Take a piece of wire and make a large circle
      :parameters (?w - wire ?p - player ?s - snare) 
      :precondition (and (inventory ?p ?w))
      :effect (and (not (inventory ?p ?w)) (inventory ?p ?s) (not (hanged ?s)))
   )

   (:action make_horizontal_bar ; Make a horizontal bar with a branch to hang your snare
      :parameters (?b - branch ?p - player ?bar - bar) 
      :precondition (and (inventory ?p ?b))
      :effect (and (not (inventory ?p ?b)) (inventory ?p ?bar))
   )

   (:action hang_snare ; Hang the circle of the snare over a path
      :parameters (?s - snare ?bar - bar ?p - player ?l - location) 
      :precondition (and (inventory ?p ?s) (inventory ?p ?bar) (not (hanged ?s)) (at ?p ?l) (made_by_animal ?l))
      :effect (and (hanged ?s) (at ?s ?l))
   )

)
"""

# Write the domain to a PDDL file
domain_filename = "domain-survive_in_the_woods.pddl"
write_to_file(domain, path, domain_filename)

## Problem definition - Building a Shelter (Step 2)


First we will implement the building a shelter section.

In [ ]:
shelter_problem = """
(define (problem build_shelter)
   (:domain survive_in_the_woods)

   (:objects
      npc - player
      waterfall camp path cliff mountain east_plain west_plain - location
      in out north south east west up down - direction
      water - water
      branch - branch
      branches - branches
      twigs - twigs
      leaves - leaves
      tree - tree
      beam -  beam
      frame - frame
      shelter - shelter
   )

   (:init
      (connected camp west path)
      (connected path east camp)
      (connected path west cliff)
      (connected cliff east path)
      (connected cliff up waterfall)
      (connected waterfall down cliff)
      (connected waterfall east mountain)
      (connected mountain west waterfall)
      (connected mountain east west_plain)
      (connected west_plain west mountain)
      (connected west_plain east east_plain)
      (connected east_plain west west_plain)
      (at npc camp)
      (at canteen camp)
      (has_water_source waterfall)
      (at branch mountain)
      (long branch)
      (thick branch)
      (at branches east_plain)
      (at twigs east_plain)
      (at leaves east_plain)
      (has_bug leaves)
      (at tree west_plain)
      (has_split_in_trunk tree)
      (dry west_plain)
      (flat west_plain)
      
   )

   (:goal (and (at shelter west_plain) (at npc west_plain)))
)
"""

# Write the problem to a PDDL file
problem_filename = "problem-build_shelter.pddl"
write_to_file(shelter_problem, path, problem_filename)

In [ ]:
!python -B pddl-parser/PDDL.py '{path}/{domain_filename}' '{path}/{problem_filename}'

----------------------------
['define',
 ['domain', 'survive_in_the_woods'],
 [':requirements', ':strips', ':typing'],
 [':types',
  'water',
  '-',
  'item',
  'tree',
  '-',
  'item',
  'branch',
  '-',
  'item',
  'branches',
  '-',
  'item',
  'twigs',
  '-',
  'item',
  'leaves',
  '-',
  'item',
  'beam',
  '-',
  'item',
  'frame',
  '-',
  'item',
  'shelter',
  '-',
  'item',
  'flowers',
  '-',
  'item',
  'wire',
  '-',
  'item',
  'snare',
  '-',
  'item',
  'bar',
  '-',
  'item',
  'player',
  'direction',
  'location'],
 [':predicates',
  ['has_water_source', '?loc', '-', 'location'],
  ['treated', '?water', '-', 'water'],
  ['at', '?obj', '-', 'object', '?loc', '-', 'location'],
  ['inventory', '?player', '-', 'player', '?item', '-', 'item'],
  ['connected',
   '?loc1',
   '-',
   'location',
   '?dir',
   '-',
   'direction',
   '?loc2',
   '-',
   'location'],
  ['blocked',
   '?loc1',
   '-',
   'location',
   '?dir',
   '-',
   'direction',
   '?loc2',
   '-',
   'l

In [ ]:
!python -B pddl-parser/planner.py '{path}/{domain_filename}' '{path}/{problem_filename}'

Time: 1.2453603744506836s
plan:
go west npc camp path
go west npc path cliff
go up npc cliff waterfall
go east npc waterfall mountain
get branch npc mountain
go east npc mountain west_plain
go east npc west_plain east_plain
get leaves npc east_plain
get twigs npc east_plain
get branches npc east_plain
go west npc east_plain west_plain
clear_spider leaves npc
insert_beam branch tree npc west_plain beam
lean_branches branches npc west_plain beam frame
layer twigs leaves npc west_plain frame shelter


## Problem deinition - Find food (Step 4, 5)

We will implement step 4 "Forage for edible wild plants" and Step 5 "Build a snare if you have string or wire".

### Implement the problem of "Forage for edible wild plants"

In [ ]:
# Implement the problem of "Forage for edible wild plants"
eat_plants = """
(define (problem eat_plants)
   (:domain survive_in_the_woods)

   (:objects
      npc - player
      waterfall camp path cliff mountain east_plain west_plain - location
      in out north south east west up down - direction
      water - water
      leaves - leaves
      flowers - flowers
   )

   (:init
      (connected camp west path)
      (connected path east camp)
      (connected path west cliff)
      (connected cliff east path)
      (connected cliff up waterfall)
      (connected waterfall down cliff)
      (connected waterfall east mountain)
      (connected mountain west waterfall)
      (connected mountain east west_plain)
      (connected west_plain west mountain)
      (connected west_plain east east_plain)
      (connected east_plain west west_plain)
      (at npc camp)
      (at canteen camp)
      (has_water_source waterfall)
      (at flowers mountain)
      (at leaves east_plain)
      (edible leaves) 
      (edible flowers) 
      (unwashed leaves)
      (unwashed flowers)
      (hungry npc)     
   )

   (:goal (and (not (hungry npc))))
)
"""

# Write the problem to a PDDL file
problem_filename = "problem-eat_plants.pddl"
write_to_file(eat_plants, path, problem_filename)

In [ ]:
!python -B pddl-parser/PDDL.py '{path}/{domain_filename}' '{path}/{problem_filename}'

In [ ]:
!python -B pddl-parser/planner.py '{path}/{domain_filename}' '{path}/{problem_filename}'

Time: 0.01492619514465332s
plan:
go west npc camp path
go west npc path cliff
go up npc cliff waterfall
get_water npc waterfall water
go east npc waterfall mountain
get flowers npc mountain
wash flowers npc water
eat flowers npc


### Implement the problem of "Build a snare if you have string or wire"

In [ ]:
# Implement the problem of "Build a snare if you have string or wire"
build_snare = """
(define (problem build_snare)
   (:domain survive_in_the_woods)

   (:objects
      npc - player
      waterfall camp path cliff mountain east_plain west_plain footpath - location
      in out north south east west up down - direction
      branch - branch
      wire - wire
      snare - snare
      bar - bar
   )

   (:init
      (connected camp west path)
      (connected path east camp)
      (connected path west cliff)
      (connected cliff east path)
      (connected cliff up waterfall)
      (connected waterfall down cliff)
      (connected waterfall east mountain)
      (connected mountain west waterfall)
      (connected mountain east west_plain)
      (connected west_plain west mountain)
      (connected west_plain east east_plain)
      (connected east_plain west west_plain)
      (connected waterfall west footpath)
      (connected footpath east waterfall)
      (at npc camp)
      (at canteen camp)
      (has_water_source waterfall)
      (at flowers mountain)
      (at leaves east_plain)
      (at branch mountain)
      (at wire path)
      (made_by_animal footpath)
   )

   (:goal (and (hanged snare) (at snare footpath)))
)
"""

# Write the problem to a PDDL file
problem_filename = "problem-build_snare.pddl"
write_to_file(build_snare, path, problem_filename)

In [ ]:
!python -B pddl-parser/PDDL.py '{path}/{domain_filename}' '{path}/{problem_filename}'

In [ ]:
!python -B pddl-parser/planner.py '{path}/{domain_filename}' '{path}/{problem_filename}'

Time: 0.06943082809448242s
plan:
go west npc camp path
get wire npc path
go west npc path cliff
go up npc cliff waterfall
go east npc waterfall mountain
get branch npc mountain
go west npc mountain waterfall
go west npc waterfall footpath
make_snare wire npc snare
make_horizontal_bar branch npc bar
hang_snare snare bar npc footpath


### DISCARDED

In [ ]:
# Deprecated
# hunt_domain = 
"""
(define (domain survive_in_the_woods)
   (:requirements :strips :typing)
   (:types
       water - item
       branch - item 
       twigs - item
       sword - item
       player direction location
   )

   (:predicates
      (has_water_source ?loc - location) ; this location has a source of fresh water.
      (treated ?water - water) ; True if the water has been decontaimated by boiling it
      (at ?obj - object ?loc - location) ; an object is at a location 
      (inventory ?player ?item) ; an item is in the player's inventory
      (connected ?loc1 - location ?dir - direction ?loc2 - location) ; location 1 is connected to location 2 in the direction
      (blocked ?loc1 - location ?dir - direction ?loc2 - location) ; the connection between location 1 and 2 in currently blocked
   )

   (:action go ; navigate to an adjacent location 
      :parameters (?dir - direction ?p - player ?l1 - location ?l2 - location) 
      :precondition (and (at ?p ?l1) (connected ?l1 ?dir ?l2) (not (blocked ?l1 ?dir ?l2)))
      :effect (and (at ?p ?l2) (not (at ?p ?l1)))
   )

   (:action get ; pick up an item and put it in the inventory
      :parameters (?item - item ?p - player ?l1 - location) 
      :precondition (and (at ?p ?l1) (at ?item ?l1))
      :effect (and (inventory ?p ?item) (not (at ?item ?l1)))
   )

   (:action drop ; drop down an item and put it in the inventory
      :parameters (?item - item ?p - player ?l1 - location) 
      :precondition (and (at ?p ?l1) (inventory ?p ?item) (not (at ?item ?l1)))
      :effect (and (not (inventory ?p ?item)) (at ?item ?l1))
   )

   (:action get_water ; get water from a location that has a water source like a lake.
      :parameters (?p - player ?loc - location ?water - water) 
      :precondition (and (at ?p ?loc) (has_water_source ?loc))
      :effect (and (inventory ?p ?water) (not (treated ?water)))
   )

   (:action build_shelter
      :parameters (?p - player ?loc - location ?branch - branch ?twigs - twigs ?shelter - shelter)
      :precondition (and (at ?branch ?loc) (at ?p ?loc) (at ?twigs ?loc))
      :effect (and (at ?shelter ?loc))
   )

   (:action sharpen
      :parameters (?p - player ?loc - location ?whetstone - whetstone ?sword - sword)
      :precondition (and (at ?p ?loc) (at ?whetstone ?loc) (at ?sword ?loc))
      :effect (and (inventory ?p sharp_sword))
   )

   (:action hunt
      :parameters (?p - player ?loc - location ?boar - boar ?sword - sharp_sword)
      :precondition (and (at ?p ?loc) (at ?boar ?loc) (inventory ?p ?sword))
      :effect (and (inventory ?p boar_meat))
   )
)
"""

# Write the domain to a PDDL file
# domain_filename = "domain-hunt.pddl"
# write_to_file(hunt_domain, path, domain_filename)

In [ ]:
# Deprecated
# shelter_problem = 
"""
(define (problem build_shelter)
   (:domain survive_in_the_woods)

   (:objects
      npc - player
      waterfall camp path cliff mountain east_plain west_plain ancient_remaining - location
      in out north south east west up down - direction
      water - water
      branch - branch
      branchB - branch
      shelter - shelter
      twigs - twigs
      sword - sword
      sharp_sword - sharp_sword
      whetstone - whetstone
      boar - boar
      boar_meat - meat
   )

   (:init
      (connected waterfall in ancient_remaining)
      (connected ancient_remaining out waterfall)

      (connected camp west path)
      (connected path east camp)
      (connected path west cliff)
      (connected cliff east path)
      (connected cliff up waterfall)
      (connected waterfall down cliff)
      (connected waterfall east mountain)
      (connected mountain west waterfall)
      (connected mountain east west_plain)
      (connected west_plain west mountain)
      (connected west_plain east east_plain)
      (connected east_plain west west_plain)
      (at npc camp)
      (at canteen camp)
      (at branch mountain)
      (at twigs east_plain)
      (at boar west_plain)
      (at sword ancient_remaining)
      (at whetstone camp)
      (has_water_source waterfall)
   )

   (:goal (and (at npc camp) (inventory npc boar_meat)))
)
"""

# Write the problem to a PDDL file
# problem_filename = "problem-hunt.pddl"
# write_to_file(shelter_problem, path, problem_filename)